In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import re

In [4]:
varname = 'ivt' ## 'ivt' or '700z'
model = 'CCSM'
### Constants ###
outdir = '/expanse/nfs/cw3e/cwp140/preprocessed/SEAK-WRF/{0}/{1}/'.format(model, varname)
datadir = '/expanse/nfs/cw3e/cwp140/preprocessed/SEAK-WRF/{0}/{1}/'.format(model, varname)
fmt = '.nc'

print('Step 1: Reading data...')
filename_pattern = datadir + 'WRFDS_{0}_*.nc'.format(varname)
print(filename_pattern)
ds = xr.open_mfdataset(filename_pattern,
                       engine='netcdf4',
                       combine='by_coords')

print('ds size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))
ds = ds.sortby('Time')
ds

Step 1: Reading data...
/expanse/nfs/cw3e/cwp140/preprocessed/SEAK-WRF/CCSM/ivt/WRFDS_ivt_*.nc
ds size in GB 25.35



<xarray.Dataset>
Dimensions:  (Time: 11315, south_north: 250, west_east: 320)
Coordinates:
    XLAT     (Time, south_north, west_east) float32 dask.array<chunksize=(365, 250, 320), meta=np.ndarray>
    XLONG    (Time, south_north, west_east) float32 dask.array<chunksize=(365, 250, 320), meta=np.ndarray>
  * Time     (Time) datetime64[ns] 2030-01-01 2030-01-02 ... 2060-12-31
Dimensions without coordinates: south_north, west_east
Data variables:
    ivtu     (Time, south_north, west_east) float32 dask.array<chunksize=(365, 250, 320), meta=np.ndarray>
    ivtv     (Time, south_north, west_east) float32 dask.array<chunksize=(365, 250, 320), meta=np.ndarray>
    ivt      (Time, south_north, west_east) float32 dask.array<chunksize=(365, 250, 320), meta=np.ndarray>
    lat      (Time, south_north, west_east) float32 dask.array<chunksize=(183, 125, 160), meta=np.ndarray>
    lon      (Time, south_north, west_east) float32 dask.array<chunksize=(183, 125, 160), meta=np.ndarray>

In [5]:
ds.Time

<xarray.DataArray 'Time' (Time: 11315)>
array(['2030-01-01T00:00:00.000000000', '2030-01-02T00:00:00.000000000',
       '2030-01-03T00:00:00.000000000', ..., '2060-12-29T00:00:00.000000000',
       '2060-12-30T00:00:00.000000000', '2060-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * Time     (Time) datetime64[ns] 2030-01-01 2030-01-02 ... 2060-12-31

In [6]:
clim_path = outdir + 'filtered_daily_mean_clim_' + varname + fmt
filtered_clim = xr.open_dataset(clim_path)
filtered_clim

<xarray.Dataset>
Dimensions:    (dayofyear: 366, south_north: 250, west_east: 320)
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366
Dimensions without coordinates: south_north, west_east
Data variables:
    ivtu       (south_north, west_east, dayofyear) float64 ...
    ivtv       (south_north, west_east, dayofyear) float64 ...
    ivt        (south_north, west_east, dayofyear) float64 ...
    lat        (south_north, west_east, dayofyear) float64 ...
    lon        (south_north, west_east, dayofyear) float64 ...

In [8]:
anomalies = ds.unify_chunks().groupby('Time.dayofyear') - filtered_clim
anomalies

/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 31 times more chunks
  return self.array[key]


<xarray.Dataset>
Dimensions:    (Time: 11315, south_north: 250, west_east: 320)
Coordinates:
    XLAT       (Time, south_north, west_east) float32 dask.array<chunksize=(183, 125, 160), meta=np.ndarray>
    XLONG      (Time, south_north, west_east) float32 dask.array<chunksize=(183, 125, 160), meta=np.ndarray>
  * Time       (Time) datetime64[ns] 2030-01-01 2030-01-02 ... 2060-12-31
    dayofyear  (Time) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365 366
Dimensions without coordinates: south_north, west_east
Data variables:
    ivtu       (Time, south_north, west_east) float64 dask.array<chunksize=(1, 125, 160), meta=np.ndarray>
    ivtv       (Time, south_north, west_east) float64 dask.array<chunksize=(1, 125, 160), meta=np.ndarray>
    ivt        (Time, south_north, west_east) float64 dask.array<chunksize=(1, 125, 160), meta=np.ndarray>
    lat        (Time, south_north, west_east) float64 dask.array<chunksize=(1, 125, 160), meta=np.ndarray>
    lon        (Time, south_north, west_east) float64 dask.array<chunksize=(1, 125, 160), meta=np.ndarray>

In [ ]:
model = 'CCSM'
path_to_wrf = '/expanse/lustre/scratch/dnash/temp_project/downloaded/WRF/'
path_to_out = '/expanse/lustre/scratch/dnash/temp_project/preprocessed/'
### START PROGRAM ###
start_yr = 2030
end_yr = 2060

## Loop through all above years
for year in np.arange(start_yr, end_yr+1):
    print('Processing ...', year)
    
    # get list of filenames that contain data from that year from current year folder
    filenames = []
    for name in glob.glob(path_to_wrf + '{0}/WRFDS_*'.format(str(year))):
        filenames.append(name)
    # sort filenames so they are in chronological order
    filenames = sorted(filenames)
    
    ds_lst = []
    for i, wrfin in enumerate(filenames):
        ds = xr.open_dataset(path_to_data + fname)

        if varname == 'ivt':
            ds = preprocess_WRF_ivt(ds, fname)
            ds_lst.append(ds)

    new_ds = xr.concat(ds_lst, dim='Time')
    
    # write to netCDF
    print('Writing', output_varname, ' to netCDF')
    fname = os.path.join(path_to_out, 'SEAK-WRF/{2}/{0}/WRFDS_{0}_{1}.nc').format(output_varname, str(year), model)
    ds.to_netcdf(path=fname, mode = 'w', format='NETCDF4')

In [5]:
path_to_data = '/expanse/lustre/scratch/dnash/temp_project/downloaded/WRF/'
fname = 'WRFDS_2030-01-01.nc'
ds = xr.open_dataset(path_to_data + fname)
ds = preprocess_WRF_ivt(ds, fname)

fname = 'WRFDS_2030-01-02.nc'
ds2 = xr.open_dataset(path_to_data + fname)
ds2 = preprocess_WRF_ivt(ds2, fname)



Masking values below surface ....
Masking values below surface ....


In [7]:
filepath = '/expanse/lustre/scratch/dnash/temp_project/preprocessed/SEAK-WRF/CCSM/ivt/'
fname = 'WRFDS_ivt_2030.nc'
ds = xr.open_dataset(filepath+fname)
ds

<xarray.Dataset>
Dimensions:  (Time: 365, south_north: 250, west_east: 320)
Coordinates:
    XTIME    (Time) float32 ...
    XLAT     (Time, south_north, west_east) float32 ...
    XLONG    (Time, south_north, west_east) float32 ...
  * Time     (Time) datetime64[ns] 2030-01-01 2030-01-02 ... 2030-12-31
Dimensions without coordinates: south_north, west_east
Data variables:
    ivtu     (Time, south_north, west_east) float32 ...
    ivtv     (Time, south_north, west_east) float32 ...
    ivt      (Time, south_north, west_east) float32 ...
    lat      (Time, south_north, west_east) float32 ...
    lon      (Time, south_north, west_east) float32 ...

In [6]:
import os, sys
import numpy as np
import itertools
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import colorsys
from matplotlib.colors import LinearSegmentedColormap # Linear interpolation for color maps
import matplotlib.patches as mpatches
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.projections import get_projection_class
import pandas as pd
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import seaborn as sns
import cmocean.cm as cmo
from datetime import timedelta
import textwrap
from matplotlib.gridspec import GridSpec
import itertools ## need this for the cbarticks

## import personal modules
sys.path.append('modules')
import cw3ecmaps as ccmaps
from plotter import draw_basemap

# Set up projection
datacrs = ccrs.PlateCarree()  ## the projection the data is in
mapcrs = ccrs.PlateCarree() ## the projection you want your map displayed in
ext = [-148., -126., 54, 61]

# Set tick/grid locations
tx = 2
ty = 2
dx = np.arange(ext[0],ext[1]+tx,tx)
dy = np.arange(ext[2],ext[3]+ty,ty)

nrows = 2
ncols = 1

## Use gridspec to set up a plot with a series of subplots that is
## n-rows by n-columns
gs = GridSpec(nrows, ncols, height_ratios=[1, 0.05], width_ratios = [1], wspace=0.01, hspace=0.2)
## use gs[rows index, columns index] to access grids

fig = plt.figure(figsize=(8, 9.))
fig.dpi = 300
fname = 'figs/test_ivt'
fmt = 'png'

ax = fig.add_subplot(gs[0, 0], projection=mapcrs)

ax = draw_basemap(ax, extent=ext, xticks=dx, yticks=dy,
                  left_lats=True, right_lats=False, bottom_lons=True)
ax.set_extent(ext, datacrs)
ax.add_feature(cfeature.STATES, edgecolor='0.4', linewidth=0.8)

## add filled contours
cmap, norm, bnds, cbarticks, cbarlbl = ccmaps.cmap('ivt') # get cmap from our custom function
cf = ax.contourf(ds.lon.values, ds.lat.values, ds.ivt.values, transform=datacrs,
                 levels=bnds, cmap=cmap, norm=norm, alpha=0.9)

# Wind barbs / vectors 
uvec_mask = ds.ivtu.where((ds.ivt >=250.)).values # mask values where IVT magnitude is less than 250 kg m-1 s-1
vvec_mask = ds.ivtv.where((ds.ivt >=250.)).values # mask values where IVT magnitude is less than 250 kg m-1 s-1

Q = ax.quiver(ds.lon.values, ds.lat.values, uvec_mask, vvec_mask, transform=datacrs, 
          color='k', regrid_shape=30,
          angles='xy', scale_units='xy', scale=500, units='xy')

# quiver key
qk = ax.quiverkey(Q, 0.8, -0.1, 250, '250 kg/m/s', labelpos='E',
                  coordinates='axes', fontproperties={'size': 6.0})

# Add color bar
cbax = plt.subplot(gs[-1, :]) # colorbar axis (last row, all columns)
cbarticks = list(itertools.compress(bnds, cbarticks)) ## this labels the cbarticks based on the cmap dictionary
cb = Colorbar(ax = cbax, mappable = cf, orientation = 'horizontal', ticklocation = 'bottom', ticks=cbarticks)
cb.set_label(cbarlbl, fontsize=11)
cb.ax.tick_params(labelsize=12)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi, transparent=True)
fig.clf()

<Figure size 2400x2700 with 0 Axes>

In [4]:
# path_to_data = '/data/downloaded/Forecasts/MPAS/combined_ivt/combined_year/'
# fname = 'ivt_mpas_current_1988.nc'
# ds = xr.open_dataset(path_to_data + fname)
# ds

<xarray.Dataset>
Dimensions:  (ens: 1, time: 1, lev: 1636, lat: 141, lon: 301)
Coordinates:
  * lon      (lon) float64 120.0 120.5 121.0 121.5 ... 268.5 269.0 269.5 270.0
  * lat      (lat) float64 10.0 10.5 11.0 11.5 12.0 ... 78.0 78.5 79.0 79.5 80.0
  * lev      (lev) float64 1.0 2.0 3.0 4.0 ... 1.634e+03 1.635e+03 1.636e+03
  * time     (time) datetime64[ns] 1988-04-01
  * ens      (ens) int32 0
Data variables:
    ivt      (ens, time, lev, lat, lon) float32 ...
    ivtx     (ens, time, lev, lat, lon) float32 ...
    ivty     (ens, time, lev, lat, lon) float32 ...